# 4. データサイエンティストによる実験パイプラインの実行
作成した.pyモジュールをCodeCommitにpushし、実験パイプラインを起動するためにexperiment.ymlを作成し、pushします。

## 1. モジュールのpush
TODO： experiment.yml以外では実験パイプラインは起動しない想定。要修正

In [ ]:
import boto3

codecommit = boto3.client('codecommit')

2回目以降のpushでは、親のCommitIdが必要になる。

### parent_commit_id取得
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/codecommit.html#CodeCommit.Client.get_file

In [ ]:
response = codecommit.get_file(
    repositoryName='demo-exp-project1',
    #commitSpecifier='string',
    filePath='preprocess.py'
)

In [ ]:
print(response['commitId'])

In [ ]:
response = codecommit.put_file(
    repositoryName='demo-exp-project1',
    branchName='main',
    fileContent=open("preprocess.py", 'rb').read(),
    filePath='preprocess.py',
    commitMessage='string',
    parentCommitId=response['commitId'],
)

In [ ]:
response = codecommit.put_file(
    repositoryName='demo-exp-project1',
    branchName='main',
    fileContent=open("train.py", 'rb').read(),
    filePath='train.py',
    parentCommitId=response['commitId'],
    commitMessage='string',
)

In [ ]:
response = codecommit.put_file(
    repositoryName='demo-exp-project1',
    branchName='main',
    fileContent=open("predict.py", 'rb').read(),
    filePath='predict.py',
    parentCommitId=response['commitId'],
    commitMessage='string',
)

In [ ]:
response = codecommit.put_file(
    repositoryName='demo-exp-project1',
    branchName='main',
    fileContent=open("evaluate.py", 'rb').read(),
    filePath='evaluate.py',
    parentCommitId=response['commitId'],
    commitMessage='string',
)

## 2. experiment.ymlのpushをトリガとする実験パイプラインの実行
実験のパラメータを記載したexperiment.ymlファイルを作成し、pushします。

### 2-1. experiment.ymlの作成

stateMachineArnには、02_create_exp_pipeline.ipynbで作成した、StepFunctionsの実験パイプラインのARNを記載します。データサイエンティストがMLOpsエンジニアから指定されるという運用を想定しています。

今回の実験では、SageMakerのビルトインコンテナを利用します。
ImageUriはリージョンによってURIが異なるので、ご自身のリージョンに適したURIを記入してください。

https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

カスタムコンテナを指定することも可能です。

In [ ]:
%%writefile experiment.yml

comment: "これはコメントです"

pipeline:
  stateMachineArn: "arn:aws:states:<your-region>:<your-account>:stateMachine:exp-preprocess-train-predict-evaluate2" # MLOpsエンジニアから指定
  input_data_uri: "s3://demo-exp-pipeline-project1/dataset/census-income.csv"
    
preprocess: 
  code: "preprocess.py"
  ImageUri: "354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # TOKYOリージョン
  #ImageUri: "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # バージニア北部リージョン
  InstanceCount: 1
  InstanceType: "ml.t3.medium"
  VolumeSizeInGB: 10
  args: [
    "--train-test-split-ratio", "0.8",
    "--train-test-split-ratio2", "0.99"
  ]

train:
  code: "train.py"
  ImageUri: "354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # TOKYOリージョン
  #ImageUri: "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # バージニア北部リージョン
  InstanceCount: 1
  InstanceType: "ml.t3.large"
  VolumeSizeInGB: 10
  args: [
    "--c", "1.0"
  ]
  
predict: 
  code: "predict.py"
  ImageUri: "354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # TOKYOリージョン
  #ImageUri: "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # バージニア北部リージョン
  InstanceCount: 1
  InstanceType: "ml.t3.xlarge"
  VolumeSizeInGB: 10
  args: [
    "--c", "1.0"
  ]
  
evaluate: 
  code: "evaluate.py"
  ImageUri: "354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # TOKYOリージョン
  #ImageUri: "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3" # バージニア北部リージョン
  InstanceCount: 1
  InstanceType: "ml.t3.2xlarge"
  VolumeSizeInGB: 10
  args: [
    "--c", "1.0"
  ]

StepFunctionsのARNの、リージョンとアカウントを置換します。

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
print(role)
print(region)
print(account_id)

In [ ]:
import textfile
 
textfile.replace('./experiment.yml', '<your-region>', region)
textfile.replace('./experiment.yml', '<your-account>', account_id)

### 2-1. experiment.ymlのpush
実験パイプラインが起動します

In [ ]:
response = codecommit.put_file(
    repositoryName='demo-exp-project1',
    branchName='main',
    fileContent=open("experiment.yml", 'rb').read(),
    filePath='experiment.yml',
    parentCommitId=response['commitId'],
    commitMessage='string',
)

StepFunctionsのコンソール画面から、実験パイプラインの実行を確認してみましょう。

実験パイプラインを用いることで、コードのモジュール化を促すことができました。モジュールとして管理することで、再利用性があがり、品質も上げることができます。
本番化やリファクタリングはMLエンジニアによって必要になりますが、ノートブックを渡すよりも効率的に本番化が実施できます。
データサイエンティストにとっては、実験の記録をとっておくことで、振り返りがしやすくなります。
管理者にとっては、記録をきちんと取っておくことで、ノウハウの蓄積、共有が行えます。

## 今後に向けて：パイプラインの改善
今回はSageMaker Processingジョブで構成された実験パイプラインを用いて、ノートブックをモジュール化し、記録をとる例を提示しました。
実験パイプラインの改善に向けては、以下のようなことができます。

* SageMaker Trainingジョブを実験パイプラインに組み込む
* SageMaker ExperimentsをStateMachine上で指定し、実験の記録を行う
* 記録した実験結果をS3に出力し、QuickSightで可視化する

SageMaker Experimentsは、StepFunctionsのステートマシンで記述を追加することで、実験結果を記録することができます。データサイエンティストがソースコードに記載しなくても実験結果を記録できるのはメリットの1つです。